In [1]:
import numpy as np
from qp.metrics.point_estimate_metric_classes import PointSigmaIQR

In [2]:
# Generate the random numbers 
SEED = 1002330
rng = np.random.default_rng(SEED)

chunk_size = 10_000
n_chunk = 10
total_size = n_chunk*chunk_size

estimate = rng.lognormal(mean=1.0, sigma=2, size=total_size)
reference = rng.lognormal(mean=1.3, sigma=1.9, size=total_size)

In [3]:
# Do the explicit full calculation
PointSigmaIQR().evaluate(estimate, reference)

1.844492171486455

In [4]:
#generator that yields chunks from estimate and reference
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# create an iterator that yields chunks of chunk_size elements
estimate_chunks = chunker(estimate, chunk_size)
reference_chunks = chunker(reference, chunk_size)

In [5]:
# A function to pass to MPI
def mpi_example(chunk):
    centroids = chunk[0].accumulate(chunk[1], chunk[2])
    return centroids

In [6]:
# Set up the data for ipyparallel
estimator = PointSigmaIQR()
estimator_list = [estimator]*n_chunk
data_chunks = [chunk for chunk in zip(estimator_list, estimate_chunks, reference_chunks)]

In [7]:
# request an MPI cluster with 5 engines
import ipyparallel as ipp
import time

In [8]:
with ipp.Cluster(controller_ip="*", engines="mpi", n=4) as rc:
    # get a broadcast_view on the cluster which is best
    # suited for MPI style computation
    view = rc.load_balanced_view()
    # run the mpi_example function on all engines in parallel
    asyncresult = view.map_async(mpi_example, data_chunks)
    # Retrieve and print the result from the engines
    asyncresult.wait_interactive()
    # retrieve actual results
    result = asyncresult.get()
    # get and print the results
    for i, res in enumerate(result):
        np.array(res)
        print(f"{i} : {res.shape}")
    final = estimator.finalize(centroids=result)
    print(final)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
mpi_example: 100%|██████████| 10/10 [00:00<00:00, 13.42tasks/s]
0 : (53, 2)
1 : (53, 2)
2 : (54, 2)
3 : (53, 2)
4 : (55, 2)
5 : (53, 2)
6 : (52, 2)
7 : (55, 2)
8 : (51, 2)
9 : (54, 2)
1.98106779379963
Stopping engine(s): 1707869025
engine set stopped 1707869025: {'exit_code': 0, 'pid': 27802, 'identifier': 'ipengine-1707869024-vf70-1707869025-27783'}
Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 27789, 'identifier': 'ipcontroller-1707869024-vf70-27783'}
